# Vertex SDK Custom Training with Python Package for TF-Keras Text Classification GPU

## Setup

In [ ]:
PROJECT_ID="aiplatform-dev"
BUCKET_NAME="gs://aiplatform-dev"
REGION="us-central1"

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

In [ ]:
! gsutil ls -al $BUCKET_NAME

In [ ]:
tutorial_name = "tf-keras-txt-cls-gpu"

## Local Training

In [ ]:
! ls trainer

In [ ]:
! cat trainer/requirements.txt

In [ ]:
! pip install -r trainer/requirements.txt

In [ ]:
! cat trainer/task.py

In [ ]:
%run trainer/task.py \
  --epochs 5

In [ ]:
! ls ./tmp

In [ ]:
! rm -rf ./tmp

## Vertex SDK Custom Training using Python Package

### Built Python Package

In [ ]:
! cat setup.py

In [ ]:
%run setup.py sdist --formats=gztar

In [ ]:
! ls dist/trainer-0.1.tar.gz

### Upload the Package to GCS

In [ ]:
python_package_gcs_uri = f"{BUCKET_NAME}/{tutorial_name}/trainer-0.1.tar.gz"
python_module_name = "trainer.task"

In [ ]:
! gsutil cp dist/trainer-0.1.tar.gz $python_package_gcs_uri

In [ ]:
! gsutil ls $python_package_gcs_uri


In [ ]:
! rm -rf dist
! rm -rf trainer.egg-info

### Pre-Built Tensorflow Container for Training

In [ ]:
prebuilt_container_image_uri = "us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-4:latest"

### Configs

In [ ]:
!cat requirements.txt

In [ ]:
!pip install -r requirements.txt

In [ ]:
from google.cloud import aiplatform

aiplatform.init(
    project=PROJECT_ID,
    staging_bucket=BUCKET_NAME,
    location=REGION,
)

In [ ]:
display_name = tutorial_name
gcs_output_uri_prefix = f"{BUCKET_NAME}/{display_name}"

replica_count = 1
machine_type = "n1-standard-32"
accelerator_count = 4
accelerator_type = "NVIDIA_TESLA_P100"

container_args = [
    '--epochs', '5'
]

### Run a CustomJob

In [ ]:
custom_training_job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=display_name,
    python_package_gcs_uri=python_package_gcs_uri,
    python_module_name=python_module_name,
    container_uri=prebuilt_container_image_uri,
)

In [ ]:
custom_training_job.run(
    base_output_dir=gcs_output_uri_prefix,
    args=container_args,
    replica_count=replica_count,
    machine_type=machine_type,
    accelerator_count=accelerator_count,
    accelerator_type=accelerator_type,
)

In [ ]:
print(f'Custom Training Job Name: {custom_training_job.resource_name}')
print(f'GCS Output URI Prefix: {gcs_output_uri_prefix}')

### Training Artifact

In [ ]:
! gsutil ls $gcs_output_uri_prefix

In [ ]:
! gsutil rm -rf $gcs_output_uri_prefix
! gsutil rm -rf $python_package_gcs_uri
